# Extracting Text

In [1]:
import fitz  # PyMuPDF is imported as fitz

In [2]:
# Open the PDF document
def extract_text_from_pdf(pdf_file_path):
    test_doc = fitz.open(pdf_file_path)  # change the path to the pdf file

    # Define thresholds to extract headers and footers
    header_threshold = 0.15  # Top 15% of the page height
    footer_threshold = 0.925  # Bottom 7.5% of the page height

    ## EXTRACT THE TEXT BY BLOCKS

    # Initialize a variable to hold the extracted text
    extracted_text_block = []
    text_string = ""

    # Loop through each page
    for page_num in range(len(test_doc)):
        page = test_doc.load_page(page_num)  # Load the page
        text_block = page.get_text("blocks")  # Extract text from the page as blocks
        page_height = page.rect.height  # Get the height of the page

        for block in text_block:
            # Get the vertical position of the block (block[1] is the top y-coordinate, block[3] is the bottom y-coordinate)
            block_top = block[1]
            block_bottom = block[3]

            # Filter out blocks that are within the header or footer region
            if (block_top > page_height * header_threshold) and (
                block_bottom < page_height * footer_threshold
            ):
                text = block[4]  # The text content is in the 5th element of the block tuple
                text_string += text + "\n\n"  # Write the block text
    # Close the document
    test_doc.close()
    return text_string

In [31]:
original_minute = r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\cv-tech-llm\DATA\documents\mayo\Acta Sesión Conmemorativa  No. 064 2024-05-24.pdf"

extracted_minute = extract_text_from_pdf(original_minute)
print(extracted_minute)

SESIÓN No. 064 CONMEMORATIVA DE CONCEJO DE 24 DE MAYO DE 2024 


 


SUMARIO: 


CAPÍTULOS   
 
 
TEMA 


 


I. 
Verificación del cuórum. 


 


II. 
Instalación de la sesión. 


 


III. 
Lectura del Orden del Día.   


 


IV. 
Himno Nacional de la República del Ecuador.  


 


V. 
Discurso de Orden en conmemoración al 24 de Mayo, día de la 
Batalla de Pichincha de 1822, a cargo de la Concejala del Distrito 
Metropolitano de Quito, Estefanía Grunauer Reinoso.  


 


VI. 
Intervención de la Alcaldesa del Distrito Metropolitano de Quito, 
Subrogante, María Fernanda Racines Corredores; 


 


VII. 
Himno a San Francisco de Quito; 


 


VIII. 
Clausura de la sesión. 


 


Anexo. 


 


 


 


 


 


 


 


SESIÓN No. 064 CONMEMORATIVA DE CONCEJO DE 24 DE MAYO DE 2024 


 


ÍNDICE: 


CAPÍTULO  
 
 
    TEMA 
 
 
 
         PÁGINA 


I. 
Verificación del cuórum.  .................................................................................  1-2 


II. 
Instalación de la sesió

# Connection to OpenAI API

In [32]:
# import libraries

from openai import OpenAI
import os
from dotenv import load_dotenv


In [33]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini") 

extracted_raw_data = extracted_minute
text_content = []

# Count the number of tokens    
text_tokens = encoding.encode(extracted_raw_data)
num_tokens = len(text_tokens)
print(f"Number of tokens : {num_tokens}")

text_content.append(extracted_raw_data) #save for future processing
     
print("\n ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")  
        

Number of tokens : 5351

 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [ ]:
## max input tokens 128K 
# if needed, for a long document, it is going to be necessary to break the document in pieces and make a recursive summarization 

In [34]:
# loading API key
load_dotenv()
key_ct = os.environ["ct_api_key"]
client = OpenAI(api_key = key_ct)

### Prompt Engineering (Zero-shot)

#### Second Approach

In [35]:
# token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# Instruction text
full_instruction = """Eres un asistente que resume actas de sesiones del Concejo Metropolitano de Quito con el objetivo de entregar informacion relevante a los ciudadanos.
      - Identifica los temas principales  discutidos en la sesión
      - Enfatiza  problemas, peticiones y soluciones o acuerdos que se llegan.
      - Identifica datos cuantitativos concretos si se tratan en la sesion 
      - Indica los nombres y los cargos de las personas  que participan cuando sea necesario.
      - Sé imparcial
      - Envía la información con la siguiente estructura
      {
     "date" : "yyyy-mm-dd", 
     "document_id" : "acta_0xx(numero_del_acta)", 
     "act_name": "Sesion (Ordinaria, Extraordinaria o Conmemorativa) No. (número de la sesion)", 
     "content":  ["cuerpo del resumen" ],	
    } 
    """
# Encode the text
instruction_tokens = encoding.encode(full_instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")

Number of tokens: 196


### cleanse instruction

In [ ]:
full_instruction = "Has un resumen del siguiente documento:"

### Sending instructions

In [40]:

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content" : full_instruction},
        {"role":"user", "content":text_content[0]},
    ],
    temperature = 0.70,
    max_tokens = 2000,
)


In [41]:
response_summary_vers001 = response.choices[0].message.content
print(response_summary_vers001)   

{
  "date": "2024-05-24",
  "document_id": "acta_064",
  "act_name": "Sesión Conmemorativa No. 064",
  "content": [
    "La sesión conmemorativa del Concejo Metropolitano de Quito se llevó a cabo el 24 de mayo de 2024, recordando el 202 aniversario de la Batalla de Pichincha de 1822. Se verificó el cuórum, confirmando la presencia de 13 concejales de un total de 22.",
    "La concejala Estefanía Grunauer Reinoso realizó un discurso destacando la importancia de la lucha por la libertad y la igualdad, recordando que la Batalla de Pichincha fue un hito en la emancipación de los pueblos de América. Hizo énfasis en la necesidad de seguir luchando contra la desigualdad y la violencia de género.",
    "La alcaldesa subrogante, María Fernanda Racines Corredores, también intervino, resaltando la relevancia de conmemorar la historia y la participación de mujeres en la independencia. Mencionó los esfuerzos de la administración municipal para fomentar el desarrollo y la inclusión en Quito, destaca

### Opcional JSON formatting

In [ ]:

# #extract ther markdown labels

import re

def extract_markdown_content(text):
    # Regular expression to find content between triple backticks
    pattern = r"```json(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        return None

# Example usage

response_summary_vers001 = extract_markdown_content(response_summary_vers001)
print(response_summary_vers001)


# MongoDB Connection

In [42]:
import json
from pymongo import MongoClient
from dotenv import load_dotenv
load_dotenv(override=True)
db_url = os.environ["DATABASE_URL"]

In [43]:
client = MongoClient(db_url)
db = client["cv_tech"]
collection = db["Summary"]
client.admin.command('ping')

{'ok': 1}

In [ ]:
response_summary_dict = json.loads(response_summary_vers001)

In [44]:

# Convert the content to a dictionary
try:
    response_summary_dict = json.loads(response_summary_vers001)
    # Add a new key-value pair to the dictionary - CHANGE EVERYTIME
    response_summary_dict["document_url"] = "https://www7.quito.gob.ec/mdmq_ordenanzas/Administraci%C3%B3n%202023-2027/Actas%20Sesiones/2024/ACTA%2064%20CONMEMORATIVA%20DE%2024%20DE%20MAYO%20DE%202024-signed.pdf"

    print(response_summary_dict)
    
    
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

{'date': '2024-05-24', 'document_id': 'acta_064', 'act_name': 'Sesión Conmemorativa No. 064', 'content': ['La sesión conmemorativa del Concejo Metropolitano de Quito se llevó a cabo el 24 de mayo de 2024, recordando el 202 aniversario de la Batalla de Pichincha de 1822. Se verificó el cuórum, confirmando la presencia de 13 concejales de un total de 22.', 'La concejala Estefanía Grunauer Reinoso realizó un discurso destacando la importancia de la lucha por la libertad y la igualdad, recordando que la Batalla de Pichincha fue un hito en la emancipación de los pueblos de América. Hizo énfasis en la necesidad de seguir luchando contra la desigualdad y la violencia de género.', 'La alcaldesa subrogante, María Fernanda Racines Corredores, también intervino, resaltando la relevancia de conmemorar la historia y la participación de mujeres en la independencia. Mencionó los esfuerzos de la administración municipal para fomentar el desarrollo y la inclusión en Quito, destacando la aprobación de i

### Insert to MongoDB collection

In [45]:
try:
    result = collection.insert_one(response_summary_dict)
    print(f"Inserted document ID: {result.inserted_id}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Inserted document ID: 672352743d432657fd70bfdd


### Save copy to JSON data file

In [ ]:

# Write the JSON data to a new file
#CHANGE THIS FILE NAME
with open(r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\data\json_files\c_o_t_example", "w", encoding="utf-8") as json_file: #CHANGE THIS FILE NAME
    json.dump(response_summary_dict, json_file, ensure_ascii=False, indent=4)

print("Text extraction completed and saved as JSON.")